## Homework 2: Intro to bias and fairness

Download the German Credit dataset: https://archive.ics.uci.edu/dataset/144/statlog+german+credit+data
(use the “numeric” version of the data)

Implement a (logistic regression) classification pipeline using an 80/20 test split. Use a regularization value of C = 1.

Treat the 20th feature (i.e., feat[19] in the numeric data, which is
related to housing) as the “sensitive attribute” i.e., z=1 if the feature value is 1.

In [ ]:
import numpy as np
from sklearn.linear_model import LogisticRegression

german_credit = np.loadtxt('german.data-numeric')
attrs = german_credit[:, :-1] 
labels = 2 - german_credit[:, -1] # (1 = Good,  2 = Bad) -> (0=Bad, 1=good)

split_point = 800
X_train, X_test = attrs[:split_point], attrs[split_point:]
y_train, y_test = labels[:split_point], labels[split_point:]

sensitive_attribute = 19

model = LogisticRegression(C=1, max_iter=1000, random_state=42)

1. Report the prevalence in the test set.

In [ ]:
prevalence = 

2. Report the per-group prevalence for z=0 and z=1.

In [ ]:
prevalence_0 = 
prevalence_1 = 

3. What is the demographic parity (expressed as a ratio between z=0 and z=1) for your classifier on the test set?

In [ ]:
parity = 

4. Report TPR_0, TPR_1, FPR_0, and FPR_1 (see "equal opportunity" slides).

In [ ]:
TPR_0 = 
TPR_1 = 

FPR_0 = 
FPR_1 = 

5. Compute PPV_0, PPV_1, NPV_0, and NPV_1 (see "are fairness goals compatible" slides).

In [ ]:
PPV_0 = 
PPV_1 = 

NPV_0 = 
NPV_1 = 

6. Implement a "fairness through unawareness" classifier, i.e., don't use Z in your feature vector. Find the classifier coefficient which undergoes the largest (absolute value) change compared to the classifier with the feature included, and report its new coefficient.


In [ ]:
new_model = LogisticRegression(C=1, max_iter=1000, random_state=42)

new_coeff = 
biggest_change_idx = 

7. Report the demographic parity of the classifier after implementing the above intervention.

In [ ]:
new_parity = 

8. Report the Generalized False Positive Rate and Generalized False Negative Rate of your original (i.e., not the one with z excluded).


In [ ]:
GFPR = 
GFNR = 

9. (harder, 2 marks) Changing the classifier threshold (much as you would to generate an ROC curve) will change the False Positive and False Negative rates for both groups (i.e., FP_0, FP_1, FN_0, FN_1). Implement a "fairness through unawareness" classifier like you did in Question 6 but instead use feature 19 (i.e., feat[18]) as the sensitive attribute. Using this classifier, find the (non-trivial) threshold that comes closest to achieving Treatment Equality, and report the corresponding values of FP_0, FP_1, FN_0, and FN_1.

In [ ]:
sensitive_attribute = 18
new_model = LogisticRegression(C=1, max_iter=1000, random_state=42)

FP_0, FP_1, FN_0, FN_1 = 

In [ ]:
answers = {
    "Q1": prevalence,           # prevalence
    "Q2": [prevalence_0, prevalence_1],  # prevalence_0, prevalence_1
    "Q3": parity,           # parity
    "Q4": [TPR_0, TPR_1, FPR_0, FPR_0], # TPR_0, TPR_1, FPR_0, FPR_1
    "Q5": [PPV_0, PPV_1, NPV_0, NPV_1], # PPV_0, PPV_1, NPV_0, NPV_1
    "Q6": [biggest_change_idx, new_coeff], # feature index, coefficient
    "Q7": new_parity,           # parity
    "Q8": [GFPR, GFNR],  # GFPR, GFNR
    "Q9": [FP_0, FP_1, FN_0, FN_1]  # FP_0, FP_1, FN_0, FN_1
}